<a href="https://colab.research.google.com/github/w-okada/voice-changer/blob/dev/VoiceChangerDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Voice Changer (デモ版)
---

このノートはVoice ChangerをColab上で動かすデモ版です。

正式版はローカルPCのDocker上で動かすアプリケーションです。

正式版は、多くの場合より少ないタイムラグで滑らかに音声を変換できます。

詳細な使用方法はこちらの[リポジトリ](https://github.com/w-okada/voice-changer)からご確認ください。


# GPUを確認
GPUを用いたほうが高速に処理が行えます。

下記のコマンドでGPUが確認できない場合は、上のメニューから

「ランタイム」→「ランタイムの変更」→「ハードウェア アクセラレータ」

でGPUを選択してください。

In [1]:
# (1) GPUの確認
!nvidia-smi

Sat Dec  3 10:47:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    25W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 使用するモデルとコンフィグファイルの指定

使用するトレーニング済みのモデルと、トレーニングで使用したコンフィグファイルのパスを指定してください。

多くの場合はGoogle Driveに格納されているファイルを使用すると思います。その場合は、下の(2-2)のセルを実行してドライブをマウントしてください

In [2]:
# (2-1) 使用するモデルとコンフィグファイルの指定
CONFIG="/content/drive/MyDrive/VoiceChanger/config.json"
MODEL="/content/drive/MyDrive/VoiceChanger/G_326000.pth"

In [3]:
# (2-2) Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# リポジトリのクローン
リポジトリをクローンします

In [4]:
# (3) リポジトリのクローン
!git clone --depth 1 https://github.com/isletennos/MMVC_Trainer.git -b v1.3.1.5 /MMVC_Trainer
!git clone --depth 1 https://github.com/w-okada/voice-changer.git
%cd /MMVC_Trainer/monotonic_align 
!cythonize -3 -i core.pyx  &> /dev/null
!mv core.cpython-*.so  monotonic_align &> /dev/null
%cd /content/voice-changer/demo/


Cloning into '/MMVC_Trainer'...
remote: Enumerating objects: 917, done.
remote: Counting objects: 100% (917/917), done.
remote: Compressing objects: 100% (828/828), done.
remote: Total 917 (delta 3), reused 889 (delta 0), pack-reused 0
Receiving objects: 100% (917/917), 53.02 MiB | 20.41 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Cloning into 'voice-changer'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 108 (delta 15), reused 75 (delta 3), pack-reused 0
Receiving objects: 100% (108/108), 18.79 MiB | 15.05 MiB/s, done.
Resolving deltas: 100% (15/15), done.
/MMVC_Trainer/monotonic_align
/content/voice-changer/demo


# ファイルの配置
アプリケーションの挙動を記した設定ファイルをコピーします(4-1)。(4-2)はコピーした設定ファイルを表示しています。もしかしたらうまく動かないときに役立つかもしれません。

In [5]:
# (4-1) 設定ファイルの配置
!cp ../template/setting_mmvc_colab.json ../frontend/dist/assets/setting.json


In [6]:
# (4-2) 設定ファイルの確認
!cat ../frontend/dist/assets/setting.json


{
    "app_title": "voice-changer",
    "majar_mode": "colab",
    "voice_changer_server_url": "/test",
    "sample_rate": 48000,
    "buffer_size": 1024,
    "prefix_chunk_size": 48,
    "chunk_size": 48,
    "speakers": [
        {
            "id": 100,
            "name": "ずんだもん"
        },
        {
            "id": 107,
            "name": "user"
        },
        {
            "id": 101,
            "name": "そら"
        },
        {
            "id": 102,
            "name": "めたん"
        },
        {
            "id": 103,
            "name": "つむぎ"
        }
    ],
    "src_id": 107,
    "dst_id": 100,
    "vf_enable": true,
    "voice_changer_mode": "realtime",
    "gpu": 0,
    "available_gpus": [-1, 0, 1, 2, 3, 4],
    "screen": {
        "enable_screen": true,
        "backgournd_image_url": "./assets/images/bg_natural_sougen.jpg"
    },
    "avatar": {
        "enable_avatar": false,
        "motion_capture_face": false,
        "motion_capture_upperbody": false,
       

# モジュールのインストール

必要なモジュールをインストールします。

In [7]:
# (5) 設定ファイルの確認
!apt-get install -y espeak libsndfile1-dev &> /dev/null
!pip install unidecode &> /dev/null
!pip install phonemizer &> /dev/null
!pip install retry &> /dev/null
!pip install python-socketio &> /dev/null
!pip install fastapi &> /dev/null
!pip install python-multipart &> /dev/null
!pip install uvicorn &> /dev/null
!pip install websockets &> /dev/null
!pip install pyOpenSSL &> /dev/null

# サーバの起動

サーバを起動します。(6-1)

サーバの起動状況を確認します。(6-2) 

このセルは繰り返し実行することになるのでCtrl+Retでセルを実行してください。

アクセスできるようになるまで、１～２分かかるようです。コーヒーでも飲みに行きましょう。

下記のようなテキストが表示されたら起動完了です。

**`INFO:root:Loaded checkpoint ...`**

```
INFO:root:Loaded checkpoint '/content/drive/MyDrive/VoiceChanger/G_326000.pth' (iteration 1136)
VoiceChanger Initialized (GPU_NUM:1)
    PHASE1:__main__
Start MMVC SocketIO Server
    CONFIG:/content/drive/MyDrive/VoiceChanger/config.json, MODEL:/content/drive/MyDrive/VoiceChanger/G_326000.pth
DEBUG:asyncio:Using selector: EpollSelector
    Phase name:MMVCServerSIO
    PHASE3:MMVCServerSIO
INFO:root:Loaded checkpoint '/content/drive/MyDrive/VoiceChanger/G_326000.pth' (iteration 1136)
```



In [8]:
# (6-1) サーバの起動
import random
PORT = 10000 + random.randint(1, 9999)
LOG_FILE = f"LOG_FILE_{PORT}"

get_ipython().system_raw(f'python3 MMVCServerSIO.py -t MMVC -p {PORT} -c {CONFIG} -m {MODEL} --colab True >{LOG_FILE} 2>&1 &')
#print(f"PORT:{PORT}, LOG_FILE:{LOG_FILE}")

In [14]:
# (6-2) サーバの起動確認 (Ctrl+Retで実行)
!tail -20 {LOG_FILE}

    Phase name:__main__
    PHASE3:__main__
INFO:root:Loaded checkpoint '/content/drive/MyDrive/VoiceChanger/G_326000.pth' (iteration 1136)
VoiceChanger Initialized (GPU_NUM:1)
    PHASE1:__main__
Start MMVC SocketIO Server
    CONFIG:/content/drive/MyDrive/VoiceChanger/config.json, MODEL:/content/drive/MyDrive/VoiceChanger/G_326000.pth
DEBUG:asyncio:Using selector: EpollSelector
    Phase name:MMVCServerSIO
    PHASE3:MMVCServerSIO
INFO:root:Loaded checkpoint '/content/drive/MyDrive/VoiceChanger/G_326000.pth' (iteration 1136)


# プロキシを起動
ウェブサーバへのアクセスをするためのプロキシを起動します。

表示されたURLをクリックして開くと別タブでアプリが開きます。

Colabなので、ロードにある程度時間がかかります(30秒くらい)。

In [15]:
# (7) プロキシを起動
from google.colab.output import eval_js
proxy = eval_js( "google.colab.kernel.proxyPort(" + str(PORT) + ")" )
print(f"{proxy}front/")

https://3lddbr773py-496ff2e9c6d22116-11741-colab.googleusercontent.com/front/
